<a href="https://colab.research.google.com/github/cjfghk5697/AjouDeeplearning_Compete/blob/main/12/13%2004%3A21.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install timm torchmetrics

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch.optim as optim
import datetime
import csv

from torchvision.datasets import ImageFolder
from torchmetrics import AUROC, ROC

import gdown
import time
import logging
import random
import torch
from torch.autograd import Variable

from google.colab import files
from torch.optim.lr_scheduler import _LRScheduler
from torchsummary import summary
from pathlib import Path
from PIL import Image

import torchvision.models as models

from torch.utils.data.dataset import random_split

import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

import pandas as pd
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm
import argparse
import albumentations as A
import cv2
from glob import glob
from albumentations.core.transforms_interface import ImageOnlyTransform
from matplotlib import pyplot as plt
import math
import timm
from torch.utils.data import Dataset, DataLoader

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_dir = '/content/drive/MyDrive/bmd_dataset/train'
model_root = './model/'

In [5]:
class CosineAnnealingWarmupRestarts(_LRScheduler):
    """
        optimizer (Optimizer): Wrapped optimizer.
        first_cycle_steps (int): First cycle step size.
        cycle_mult(float): Cycle steps magnification. Default: -1.
        max_lr(float): First cycle's max learning rate. Default: 0.1.
        min_lr(float): Min learning rate. Default: 0.001.
        warmup_steps(int): Linear warmup step size. Default: 0.
        gamma(float): Decrease rate of max learning rate by cycle. Default: 1.
        last_epoch (int): The index of last epoch. Default: -1.
    """
    
    def __init__(self,
                 optimizer : torch.optim.Optimizer,
                 first_cycle_steps : int,
                 cycle_mult : float = 1.,
                 max_lr : float = 0.1,
                 min_lr : float = 0.001,
                 warmup_steps : int = 0,
                 gamma : float = 1.,
                 last_epoch : int = -1
        ):
        assert warmup_steps < first_cycle_steps
        
        self.first_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle_mult = cycle_mult # cycle steps magnification
        self.base_max_lr = max_lr # first max learning rate
        self.max_lr = max_lr # max learning rate in the current cycle
        self.min_lr = min_lr # min learning rate
        self.warmup_steps = warmup_steps # warmup step size
        self.gamma = gamma # decrease rate of max learning rate by cycle
        
        self.cur_cycle_steps = first_cycle_steps # first cycle step size
        self.cycle = 0 # cycle count
        self.step_in_cycle = last_epoch # step size of the current cycle
        
        super(CosineAnnealingWarmupRestarts, self).__init__(optimizer, last_epoch)
        
        # set learning rate min_lr
        self.init_lr()
    
    def init_lr(self):
        self.base_lrs = []
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = self.min_lr
            self.base_lrs.append(self.min_lr)
    
    def get_lr(self):
        if self.step_in_cycle == -1:
            return self.base_lrs
        elif self.step_in_cycle < self.warmup_steps:
            return [(self.max_lr - base_lr)*self.step_in_cycle / self.warmup_steps + base_lr for base_lr in self.base_lrs]
        else:
            return [base_lr + (self.max_lr - base_lr) \
                    * (1 + math.cos(math.pi * (self.step_in_cycle-self.warmup_steps) \
                                    / (self.cur_cycle_steps - self.warmup_steps))) / 2
                    for base_lr in self.base_lrs]

    def step(self, epoch=None):
        if epoch is None:
            epoch = self.last_epoch + 1
            self.step_in_cycle = self.step_in_cycle + 1
            if self.step_in_cycle >= self.cur_cycle_steps:
                self.cycle += 1
                self.step_in_cycle = self.step_in_cycle - self.cur_cycle_steps
                self.cur_cycle_steps = int((self.cur_cycle_steps - self.warmup_steps) * self.cycle_mult) + self.warmup_steps
        else:
            if epoch >= self.first_cycle_steps:
                if self.cycle_mult == 1.:
                    self.step_in_cycle = epoch % self.first_cycle_steps
                    self.cycle = epoch // self.first_cycle_steps
                else:
                    n = int(math.log((epoch / self.first_cycle_steps * (self.cycle_mult - 1) + 1), self.cycle_mult))
                    self.cycle = n
                    self.step_in_cycle = epoch - int(self.first_cycle_steps * (self.cycle_mult ** n - 1) / (self.cycle_mult - 1))
                    self.cur_cycle_steps = self.first_cycle_steps * self.cycle_mult ** (n)
            else:
                self.cur_cycle_steps = self.first_cycle_steps
                self.step_in_cycle = epoch
                
        self.max_lr = self.base_max_lr * (self.gamma**self.cycle)
        self.last_epoch = math.floor(epoch)
        for param_group, lr in zip(self.optimizer.param_groups, self.get_lr()):
            param_group['lr'] = lr

In [6]:
class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, degenerated_to_sgd=True):
        if not 0.0 <= lr:
            raise ValueError("Invalid learning rate: {}".format(lr))
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        
        self.degenerated_to_sgd = degenerated_to_sgd
        if isinstance(params, (list, tuple)) and len(params) > 0 and isinstance(params[0], dict):
            for param in params:
                if 'betas' in param and (param['betas'][0] != betas[0] or param['betas'][1] != betas[1]):
                    param['buffer'] = [[None, None, None] for _ in range(10)]
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay, buffer=[[None, None, None] for _ in range(10)])
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = group['buffer'][int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    elif self.degenerated_to_sgd:
                        step_size = 1.0 / (1 - beta1 ** state['step'])
                    else:
                        step_size = -1
                    buffered[2] = step_size

                # more conservative since it's an approximated value
                if N_sma >= 5:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size * group['lr'], exp_avg, denom)
                    p.data.copy_(p_data_fp32)
                elif step_size > 0:
                    if group['weight_decay'] != 0:
                        p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)
                    p_data_fp32.add_(-step_size * group['lr'], exp_avg)
                    p.data.copy_(p_data_fp32)

        return loss

In [7]:
def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# Train

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Model list

In [9]:
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = True  # type: ignore
    torch.backends.cudnn.benchmark = True  # type: ignore
seed_everything(42)

In [10]:
# all_densenet_models = timm.list_models('**')
# all_densenet_models

Model

In [11]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model('efficientnet_b5', pretrained=True)
        self.backbone.classifier = nn.Identity()
        self.fc = nn.Linear(2048, 3)

    def forward(self, x):
        return self.fc(self.backbone(x))

Making a dataset

In [12]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomCrop(224),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = ImageFolder(train_dir, transform=train_transform)

# normal: 0, osteopenia:1, osteoporosis:2

Train & Validation

In [13]:
net = Model()
net.train()
net = net.to(device)

criterion = nn.CrossEntropyLoss()

optimizer = RAdam(net.parameters(), lr=1e-2, betas=(0.9, 0.999), weight_decay=1e-2)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)    
scheduler = CosineAnnealingWarmupRestarts(optimizer,
                                          first_cycle_steps=60,
                                         cycle_mult=1.0,
                                          max_lr=0.005,
                                          min_lr=0.0001,
                                          warmup_steps=12,
                                          gamma=1.0)

No pretrained weights exist for this model. Using random initialization.


In [14]:
def save_model(model, acc, date_time, name):
    model_name = name + '.pth'
    model_path = Path(model_root + date_time)
    model_path.mkdir(parents=True, exist_ok=True)
    print('Saving model (Accuracy {:.2f}%) to {}'.format(acc*100, str(model_path / model_name)) )
    torch.save({'model_state_dict':model.state_dict(), 'acc':acc}, str(model_path / model_name) )

In [15]:
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, auc=False, device=device,scheduler=scheduler):
    tz = datetime.timezone(datetime.timedelta(hours=9)) # Timezone infomation
    date_time = datetime.datetime.now(tz).strftime('%Y-%m-%d-%H-%M-%S')
    
    best_val_acc = 0
    best_train_acc = 0
    epoch_train_acc = 0
    
    for epoch in range(num_epochs + 1):
        print('\n------------------------')
        print('EPOCH {}/{}'.format(epoch, num_epochs))
        print('------------------------')
        
        if auc == True:
            auc_roc_metric = AUROC(num_classes=3, average=None)
            roc_metric = ROC(num_classes=3)

        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0
            
            total_loss = 0
            total_size = 0
            total_corrects = 0
            
            # 학습 전 성능 확인
            if (epoch == 0) and (phase == 'train'):
                continue
            
            num_iteration = len(dataloaders_dict[phase])
            
            for idx, (inputs, labels) in enumerate(tqdm(dataloaders_dict[phase])):
                optimizer.zero_grad()



                with torch.set_grad_enabled(phase == 'train'):
                    inputs, labels = inputs.to(device), labels.to(device)
                    inputs, labels_a, labels_b, lam = mixup_data(inputs,labels)
                    inputs, labels_a, labels_b = map(Variable, (inputs, labels_a, labels_b))
                    outputs = net(inputs)
                    loss = mixup_criterion(criterion, outputs, labels_a, labels_b, lam)
                    _, preds = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    epoch_loss = loss.item()
                    total_loss += epoch_loss

                    epoch_corrects = torch.sum(preds == labels.data)
                    epoch_acc = epoch_corrects.double() / inputs.size(0)
                    total_corrects += epoch_corrects
                    total_size += inputs.size(0)
                    
                    if phase == 'train':
                        print('{} [{}/{}] LOSS: {:.4f} ACC: {:.4f}'.format(phase, idx+1, num_iteration, epoch_loss, epoch_acc))

                    if (phase == 'val') & (auc == True):
                        auc_roc_metric(outputs, labels)
                        roc_metric(outputs, labels)
            
            epoch_loss_avg = total_loss / num_iteration
            epoch_acc_avg = total_corrects / total_size
            scheduler.step()
            if phase == 'train':
                if best_train_acc < epoch_acc_avg:
                    best_train_acc = epoch_acc_avg
                print('{} LOSS: {:.4f} ACC: {:.4f} BEST ACC: {:.4f}'.format(phase, epoch_loss_avg, epoch_acc_avg, best_train_acc))

            if phase == 'val':
                if (epoch_acc_avg == best_val_acc) & (epoch_acc_avg == best_train_acc):
                    save_model(net, epoch_acc_avg, date_time, 'best')
                elif best_val_acc < epoch_acc_avg:
                    best_val_acc = epoch_acc_avg
                    print('Best Validation Accuracy: {:.4f}'.format(epoch_acc_avg))
                    save_model(net, epoch_acc_avg, date_time, 'best')

                print('{} LOSS: {:.4f} ACC: {:.4f} BEST ACC: {:.4f}\n'.format(phase, epoch_loss_avg, epoch_acc_avg, best_val_acc))

                if auc == True:
                    for auroc, fpt, tpr, thresholds in zip(auc_roc_metric.compute(), *roc_metric.compute()):
                        print('auc: {:5.2f}'.format(auroc* 100))
                        size = min(len(fpt), len(tpr))
                        plt.plot(fpt[:size].cpu(), tpr[:size].cpu())
                        plt.show()
                        plt.cla()

In [16]:
num_epochs=60

batch_size = 32
num_workers = 4
view_auc = False

train_size = int(0.7*len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=num_workers)

dataloaders_dict = {"train": train_loader, "val": val_loader}

train_model(net, dataloaders_dict, criterion, optimizer, num_epochs, view_auc,scheduler=scheduler)


------------------------
EPOCH 0/60
------------------------


100%|██████████| 29/29 [00:34<00:00,  1.20s/it]


Best Validation Accuracy: 0.2800
Saving model (Accuracy 28.00%) to model/2021-12-13-15-42-45/best.pth
val LOSS: 1.1006 ACC: 0.2800 BEST ACC: 0.2800


------------------------
EPOCH 1/60
------------------------


  0%|          | 0/66 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1050.)
  2%|▏         | 1/66 [00:06<07:12,  6.66s/it]

train [1/66] LOSS: 1.0897 ACC: 0.3750


...

train [65/66] LOSS: 0.4097 ACC: 0.3750


100%|██████████| 66/66 [00:31<00:00,  2.10it/s]


train [66/66] LOSS: 0.7664 ACC: 0.1500
train LOSS: 0.5595 ACC: 0.6862 BEST ACC: 0.7205


100%|██████████| 29/29 [00:04<00:00,  7.00it/s]

val LOSS: 1.3400 ACC: 0.3700 BEST ACC: 0.4322

